In [10]:
import re
import json


In [11]:
def extract_pdfminer1(str):

    from pdfminer.high_level import extract_text

    text = extract_text(str)
    return text

In [12]:
def extract_pdfminer2(str):
    from io import StringIO

    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfdocument import PDFDocument
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.pdfpage import PDFPage
    from pdfminer.pdfparser import PDFParser

    output_string = StringIO()
    with open(str, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser,password="AKAS2003")
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)

    return output_string.getvalue()
    

In [13]:
def extract_pypdf2(path):
    from PyPDF2 import  PdfReader
    from io import StringIO
    
    a=PdfReader(path)
    if a.is_encrypted:
        #passw=input("Enter the password")
        a.decrypt('AKAS2003')
    text=""
    for pageno in range(len(a.pages)):
        text=text+a.pages[pageno].extract_text()
    return text   
        

In [14]:
def extract_pymupdf(path):
    
    #Usinf pymupdf
    import fitz  # this is pymupdf
    #extract text page by page
    with fitz.open(path) as doc:
        pymupdf_text = ""
        if(doc.is_encrypted):
                passw=input("Enter the password")
                doc.authenticate(password=passw)
        for page in doc:
            pymupdf_text += page.get_text("Text")
        return pymupdf_text    

In [15]:
class Aadhaar:
    def __init__(self,path):
        self.txt=extract_pymupdf(path)
    def get_details(self):
        pattern=re.compile(r'Enrolment No\.: (?P<enrolment_no>[^\n]*)\nTo\n[^\n]*\n(?P<name>[^\n]*)\n[S,W]\/O: (?P<fathers_name>[^\n]*)\n(?P<address>.*)(?P<phonenumber>\d{10})\n(?P<aadhaar_number>^\d{4} \d{4} \d{4}\n).*(?P<vid>\d{4} \d{4} \d{4} \d{4})\n.*DOB: (?P<dob>[^\n]*)\n.*(?P<gender>MALE|FEMALE|Female|Male)',re.M|re.A|re.S)
       #gets all info in one match(enrolment to V) which can then be parsed by the groups
        return  pattern.search(self.txt)    
    def get_enrolment_no(self):
        return self.get_details().group('enrolment_no')
    def get_name(self):
        return self.get_details().group('name')
    def get_fathers_name(self):
        return self.get_details().group('fathers_name')
    def get_address(self):
        return self.get_details().group('address')
    def get_phonenumber(self):
        return self.get_details().group('phonenumber')
    def get_aadhaarnumber(self):
        return self.get_details().group('aadhaar_number')
    def get_vid(self):
        return self.get_details().group('vid')
    def get_gender(self):
        return self.get_details().group('gender')
    def get_dob(self):
        return self.get_dob().group('dob')
    def to_json(self,path):
        dict={'VID':self.get_vid(),'Enrolment no':self.get_enrolment_no(),'Name':self.get_name(),
      'Fathers name':self.get_fathers_name(),'Address':self.get_address(),'Phonenumber':self.get_phonenumber(),'Aadhaar number':self.get_aadhaarnumber(),'Gender':self.get_gender(),'DOB':self.get_dob()}
        with open(path,'w') as f:
            json.dump(dict,f,indent=2)
        
    
        

In [16]:
str='EAadhaar_og.pdf'

In [17]:
aad=Aadhaar(str)

Enter the passwordAKAS2003


In [ ]:
aad.to_json('newjson.json')